In [2]:
import json

In [3]:
with open('jmdict-eng-3.6.1.json', 'r', encoding='utf-8') as f:
    jmdict = json.load(f)

In [4]:
words = {}
jmdict_single = {}
c = 0
dup = 0

for char in jmdict['words']:
    kanji = char['kanji']
    if len(kanji) == 0:
        continue
    kana = char['kana']
    
    # わたし、定時で帰ります。
    kanji_text = [k['text'].removesuffix('。') for k in kanji]
    kana_text = [k['text'].removesuffix('。') for k in kana]
    if kana_text[0] == '二リンさん':
        kana_text[0] = 'にリンさん'

    # TODO: appliesToKanji
    c += 1
    # print(kanji_text, kana_text)
    
    for kanji in kanji_text:
        if kanji in words:
            words[kanji].extend(kana_text)
            dup += 1
        elif kanji in jmdict_single:
            jmdict_single[kanji].extend(kana_text)
            dup += 1
        else:
            if len(kanji) == 1:
                jmdict_single[kanji] = kana_text
            else:
                words[kanji] = kana_text

with open('jmdict.csv', 'w', encoding='utf-8') as out, open('jmdict-single.csv', 'w', encoding='utf-8') as out_single:
    for kanji in words:
        kanas = '\t'.join(words[kanji])
        out.write(f'{kanji}\t{kanas}\n')
    for kanji in jmdict_single:
        kanas = '\t'.join(jmdict_single[kanji])
        out_single.write(f'{kanji}\t{kanas}\n')

print('c', c)
print('dup', dup)

c 172429
dup 4209


In [5]:
with open('kanjidic2-en-3.6.1.json', 'r', encoding='utf-8') as f:
    kanjidic = json.load(f)

In [6]:
c = 0
with open('kanjidic.csv', 'w', encoding='utf-8') as out:
    for char in kanjidic['characters']:
        kanji = char['literal']
        assert len(kanji) == 1

        readingMeaning = char['readingMeaning']
        groups = readingMeaning['groups']
        readings = [r['value'] for group in groups for r in group['readings'] if r['type'].startswith('j')]
        nanori = readingMeaning['nanori']
        kanas = readings + nanori

        if kanji in jmdict_single:
            kanas = set(jmdict_single.pop(kanji) + kanas)
        if len(kanas) == 0:
            continue

        # TODO: Optional 送り仮名?
        kanas = [k.replace('-', '').replace('.', '') for k in kanas]

        c += 1
        # print(kanji_text, kana_text)
        
        kanas = '\t'.join(kanas)
        out.write(f'{kanji}\t{kanas}\n')
    for kanji in jmdict_single:
        kanas = jmdict_single[kanji]

        c += 1

        kanas = '\t'.join(kanas)
        out.write(f'{kanji}\t{kanas}\n')
c

10314